    
# ECON320-3 Assignment 4: James (Minkyu) Song 


In [24]:
# Script name: james_song_assignment_04.ipynb
# Purpose of script: Assignment 4 of ECON320 with Dr.O'Connell
# Author: James (Minkyu) Song
# Date Created: September 19th 2022
# --------------------------
# Notes: NA
# ---------------------------

## Part I

* Objective: Consider the properties of OLS on any sample of data.*

* Reference: Wooldridge (7th edition), Chapter 2, Section 3*

Using all of the previous up to this point, calculate TSS, ESS and RSS in your sample.

In [25]:
import math

import pandas as pd
import numpy as np

In [30]:
# creating sample, referenced from previous assignmnet submissions
# and lab lectures

birthdaystring = "04021998"
rng = np.random.RandomState(int(birthdaystring))


x = rng.normal(loc=10, scale=1 , size=1000000)
# u normal dist with mean 0 std 1 
u = rng.normal(loc=0, scale=1 , size=1000000)
# b_0 = 0+2; b_1= 4+0 
y = 2 + 4*x + u 
print("y= 2 + 4*x + u ")
data = {"x": x, "u": u, "y": y}
df = pd.DataFrame(data)
# Get a sample of 1000 observations
sample_df = df.sample(n = 1000, random_state = int(birthdaystring))
print("cov(x,y) : ", sample_df[['y', 'x']].cov().iloc[0,1])


y= 2 + 4*x + u 
cov(x,y) :  4.01532677941436



a. Calculate $R^2$, which is ESS/TSS. 

In [35]:
sample_df.head()

,x,u,y,y_pred,residuals
480891,9.320804,1.198214,40.481429,39.277208,1.204222
179895,11.362336,0.697495,48.146839,47.368491,0.778348
90252,11.387784,0.254672,47.805808,47.469350,0.336457
167375,10.930718,0.008831,45.731705,45.657844,0.073860
198512,8.520829,-1.002781,35.080534,36.106636,-1.026102


In [36]:
def sample_betas(sample_x, sample_y):
    covxy= np.cov(sample_x, sample_y)[1,0]
    print("covariance is: "+ str(covxy))
    varx= sample_x.var()
    beta1_sample = covxy/varx
    beta0_sample = sample_y.mean() - (beta1_sample* sample_x.mean())
    print(f'b0:{beta0_sample}')
    print(f'b1:{beta1_sample}')

    return beta0_sample, beta1_sample

def predict_y(sample_x, sample_y):
    beta0_sample, beta1_sample = sample_betas(sample_x, sample_y)
    y_pred = beta1_sample *sample_x +beta0_sample
    return y_pred

def residuals(sample_y, sample_y_pred):
    return sample_y - sample_y_pred



In [37]:
sample_df['y_pred']= predict_y(sample_df['x'], sample_df['y'])
sample_df['residuals'] = residuals(sample_df['y'], sample_df['y_pred'])

covariance is: 4.01532677941436
b0:2.335705145168248
b1:3.9633387052065174


In [38]:
def ex_sum_sq(y_pred):
    return (y_pred-y_pred.mean())**2

def res_sum_sq(y_column, y_pred):
    return (y_column-y_pred)**2

def total_sum_sq(y_column, y_pred):
    return (y_column - y_pred.mean())**2

def generate_sumsq_df(y_column, y_pred):
    sumsq_df = pd.DataFrame()
    ess_arr = ex_sum_sq(y_pred)
    rss_arr = res_sum_sq(y_column, y_pred)
    tss_arr = total_sum_sq(y_column, y_pred)

    data = list(zip(ess_arr, rss_arr, tss_arr, ess_arr+rss_arr))
    sumsq_df = pd.DataFrame(data= data, columns = ['ESS', 'RSS', 'TSS', 'ESS+RSS'])
    return sumsq_df

In [43]:
sumsq_df= generate_sumsq_df(sample_df['y'], sample_df['y_pred'])
for column in sumsq_df.columns[:-1]:
    print(f"{column:10}: {sumsq_df[column].sum()}" )

ESS       : 15898.18593886626
RSS       : 1056.479506583848
TSS       : 16954.66544545011


In [44]:
r_squared = sumsq_df['ESS'].sum()/sumsq_df['TSS'].sum()
print("R Squared Value is : ", r_squared)

R Squared Value is :  0.9376879768000751



b. Show that for any individual observation, $(y_i - \bar{y})^2 \neq (\hat{y}_i-\bar{y})^2 + \hat{u}_i^2$, but it works when summing components across all observations i.e., that $$TSS=ESS+RSS$$

In [45]:
sumsq_df.head()

,ESS,RSS,TSS,ESS+RSS
0,7.297028,1.450150,2.241247,8.747177
1,29.051909,0.605826,38.048301,29.657735
2,30.149337,0.113204,33.957410,30.262541
3,13.537509,0.005455,14.086479,13.542964
4,34.478889,1.052885,47.582052,35.531774


In [46]:
indiv_obvs_bool = sumsq_df['TSS']==sumsq_df['ESS+RSS']
indiv_obvs_bool.drop_duplicates(keep="first")
# after dropping all duplicates, only false is left, which means sumsq_df['TSS']==sumsq_df['ESS+RSS'] was all False.

0    False
dtype: bool

In [47]:
for column in sumsq_df.columns[2:]:
    print(f"SUM of {column:10}: {sumsq_df[column].sum()}" )

SUM of TSS       : 16954.66544545011
SUM of ESS+RSS   : 16954.665445450108



c. From the sample and estimates from the previous assignment, show that $\sum_{\forall i}{\hat{u}_i} \approx  0$ . Show that $\sum_{\forall i}({\hat{u}_i*x_i}) \approx  0$

In [48]:
sum_of_resids = abs(round(sample_df['residuals'].sum(), 3))
print(f"Sum of residuals: {sum_of_resids}")

sum_xu = round((sample_df['x']*sample_df['residuals']).sum(), 4)
print(f"Sum (x*u): {sum_xu}")

Sum of residuals: 0.0
Sum (x*u): 0.0



d. From the sample and estimates from the previous assignment, calculate the variance and standard error of $\hat{\beta_1}$. 


In [49]:
def sigma_hat(residuals):
    return (residuals**2).sum()/ (len(residuals)-2)

def var_se_sample_beta1(residuals, x_column):
    sig_hat2 = sigma_hat(residuals)
    tss_x = ex_sum_sq(x_column).sum() #ess calculation is the same as xi - xmean
    tss_x_sqrt = tss_x**(1/2) # since se(sample b_1) = sigmahat2.sqrt() / tssx.sqrt
    var_sample_b1 = (sig_hat2)/tss_x
    se_sample_b1 = (sig_hat2**(1/2)) / tss_x_sqrt
    print(f"Variance of beta1 : {var_sample_b1} and Standard Error : {se_sample_b1}")
    return var_sample_b1, se_sample_b1

variance_sample_b1, standard_error_sample_b1 = var_se_sample_beta1(sample_df['residuals'], sample_df['x'])


Variance of beta1 : 0.0010459365531175003 and Standard Error : 0.03234094236594692


In [52]:
print("variance^(1/2) - standrard error =", abs(round(variance_sample_b1**(1/2) - standard_error_sample_b1, 3)))

variance^(1/2) - standrard error = 0.0
